# Data Analysis Pipeline

## 1. Load Data

In [5]:
import pandas as pd 

In [6]:
df = pd.read_csv("../data/raw/insurance.csv")
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


### Check file type and encoding

In [23]:
pip install chardet

Note: you may need to restart the kernel to use updated packages.


In [24]:
import os
import chardet
import csv
import pandas as pd

In [25]:
file_path = "../data/raw/insurance.csv"
_, extension = os.path.splitext(file_path)
file_type = extension.lower()
f"File type: {file_type}"


'File type: .csv'

In [26]:
file_path = "../data/raw/insurance.csv"

with open(file_path, 'rb') as file:
    result = chardet.detect(file.read())
file_encoding = result['encoding']
print(f"File encoding: {file_encoding}")



File encoding: ascii


### Check delimiters

In [28]:

sample_size = 1024
with open(file_path, 'r', newline='', encoding=file_encoding) as file:
    sample = file.read(sample_size)

sniffer = csv.Sniffer()
delimiter = sniffer.sniff(sample).delimiter
print(f"Delimiter: {delimiter}")



Delimiter: ,


### Skip rows

Running dropna does not remove any rows

In [33]:
df.shape

(1338, 7)

In [34]:
df.dropna(axis=0)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,10600.54830
1334,18,female,31.920,0,no,northeast,2205.98080
1335,18,female,36.850,0,no,southeast,1629.83350
1336,21,female,25.800,0,no,southwest,2007.94500


## 2. Clean Data

### Drop unneeded columns
Drop the column "charges" as I am not using that in my analysis

In [35]:
dfdrop = df.drop(columns=['charges'])
dfdrop.head(5)

,age,sex,bmi,children,smoker,region
0,19,female,27.900,0,yes,southwest
1,18,male,33.770,1,no,southeast
2,28,male,33.000,3,no,southeast
3,33,male,22.705,0,no,northwest
4,32,male,28.880,0,no,northwest


### Check for incorrect data

Check for missing data

In [44]:
#There are no missing values

dfdrop.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
dtype: int64

Check datatypes

In [45]:
#The datatypes are ok

dfdrop.dtypes

age           int64
sex          object
bmi         float64
children      int64
smoker       object
region       object
dtype: object

Check outliers

In [43]:
#no obvious outliers

dfdrop.describe()

,age,bmi,children
count,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918
std,14.049960,6.098187,1.205493
min,18.000000,15.960000,0.000000
25%,27.000000,26.296250,0.000000
50%,39.000000,30.400000,1.000000
75%,51.000000,34.693750,2.000000
max,64.000000,53.130000,5.000000


Check unique values

In [47]:
#Everything here seems fine

dfdrop.nunique(axis=0)

age          47
sex           2
bmi         548
children      6
smoker        2
region        4
dtype: int64

## 2. Process and Wrangle Data

### Create any needed new column

In [126]:
#Added a column turning the presence of children into a binary status (yes or no).

dfdrop['has children'] = df['children'].apply(lambda x: 'yes' if x > 0 else 'no')
dfdrop

,age,sex,bmi,children,smoker,region,has_children,has children
0,19,female,27.900,0,yes,southwest,no,no
1,18,male,33.770,1,no,southeast,yes,yes
2,28,male,33.000,3,no,southeast,yes,yes
3,33,male,22.705,0,no,northwest,no,no
4,32,male,28.880,0,no,northwest,no,no
...,...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,yes,yes
1334,18,female,31.920,0,no,northeast,no,no
1335,18,female,36.850,0,no,southeast,no,no
1336,21,female,25.800,0,no,southwest,no,no


In [127]:
#accidentially created to has children columns, so drop one here

dfdrop2 = dfdrop.drop(columns=['has_children'])
dfdrop2

,age,sex,bmi,children,smoker,region,has children
0,19,female,27.900,0,yes,southwest,no
1,18,male,33.770,1,no,southeast,yes
2,28,male,33.000,3,no,southeast,yes
3,33,male,22.705,0,no,northwest,no
4,32,male,28.880,0,no,northwest,no
...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,yes
1334,18,female,31.920,0,no,northeast,no
1335,18,female,36.850,0,no,southeast,no
1336,21,female,25.800,0,no,southwest,no


In [92]:
dfdrop2['has children'].value_counts()

yes    764
no     574
Name: has children, dtype: int64

In [89]:


# Here I created a new column that classifies each row into one of the BMI categories.

def bmi_category(bmi):
    if bmi < 16:
        return 'Underweight'
    elif 16 <= bmi <= 18.4:
        return 'Underweight'
    elif 18.5 <= bmi <= 24.9:
        return 'Normal'
    elif 25 <= bmi <= 29.9:
        return 'Overweight'
    else:
        return 'Obese'

dfdrop2['BMI Category'] = dfdrop2['bmi'].apply(lambda x: bmi_category(x))

dfdrop2


,age,sex,bmi,children,smoker,region,has children,BMI Category
0,19,female,27.900,0,yes,southwest,no,Overweight
1,18,male,33.770,1,no,southeast,yes,Obese
2,28,male,33.000,3,no,southeast,yes,Obese
3,33,male,22.705,0,no,northwest,no,Normal
4,32,male,28.880,0,no,northwest,no,Overweight
...,...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,yes,Obese
1334,18,female,31.920,0,no,northeast,no,Obese
1335,18,female,36.850,0,no,southeast,no,Obese
1336,21,female,25.800,0,no,southwest,no,Overweight


In [84]:
dfdrop2['BMI Category'].value_counts()

obese          719
overweight     377
normal         222
underweight     20
Name: BMI Category, dtype: int64

In [85]:
#As there are very few entries (20) for underweight people, I am going to remove this completely. 
#After printing the new dataset, I can see that 20 rows have been removed.

filtered_df = dfdrop2[dfdrop2['bmi'] >= 18.5]
filtered_df

,age,sex,bmi,children,smoker,region,has children,BMI Category
0,19,female,27.900,0,yes,southwest,no,overweight
1,18,male,33.770,1,no,southeast,yes,obese
2,28,male,33.000,3,no,southeast,yes,obese
3,33,male,22.705,0,no,northwest,no,normal
4,32,male,28.880,0,no,northwest,no,overweight
...,...,...,...,...,...,...,...,...
1333,50,male,30.970,3,no,northwest,yes,obese
1334,18,female,31.920,0,no,northeast,no,obese
1335,18,female,36.850,0,no,southeast,no,obese
1336,21,female,25.800,0,no,southwest,no,overweight


In [86]:
#I now confirm the underweight category no longer exists.

filtered_df['BMI Category'].value_counts()

obese         719
overweight    377
normal        222
Name: BMI Category, dtype: int64

In [115]:
# Here I created a new column that classifies the entries into age groups

def age_category(age):
    if age < 19:
        return 'teen'
    elif 20 <= age <= 29:
        return '20s'
    elif 30 <= age <= 39:
        return '30s'
    elif 40 <= age <= 49:
        return '40s'
    elif 50 <= age <= 59:
        return '50s'  
    else:
        return '60s'

filtered_df['age group'] = filtered_df['age'].apply(lambda x: age_category(x))

filtered_df.head()

,age group,age,sex,children,has children,smoker,region,bmi,BMI Category
0,60s,19,female,0,NaN,yes,southwest,27.900,overweight
1,teen,18,male,1,NaN,no,southeast,33.770,obese
2,20s,28,male,3,NaN,no,southeast,33.000,obese
3,30s,33,male,0,NaN,no,northwest,22.705,normal
4,30s,32,male,0,NaN,no,northwest,28.880,overweight


In [113]:
filtered_df['age group'].value_counts()

40s     279
20s     273
50s     268
30s     252
60s     179
teen     67
Name: age group, dtype: int64

In [124]:
#here I re-arranged the order of the columns

new_order = ['age group', 'age', 'sex', 'children', 'has children', 'smoker', 'region', 'bmi', 'BMI Category']
filered_df = df.reindex(columns=new_order)
filtered_df

,age group,age,gender,children,has children,smoker,region,bmi,BMI Category
0,60s,19,female,0,no,yes,southwest,27.900,overweight
1,teen,18,male,1,yes,no,southeast,33.770,obese
2,20s,28,male,3,yes,no,southeast,33.000,obese
3,30s,33,male,0,no,no,northwest,22.705,normal
4,30s,32,male,0,no,no,northwest,28.880,overweight
...,...,...,...,...,...,...,...,...,...
1333,50s,50,male,3,yes,no,northwest,30.970,obese
1334,teen,18,female,0,no,no,northeast,31.920,obese
1335,teen,18,female,0,no,no,southeast,36.850,obese
1336,20s,21,female,0,no,no,southwest,25.800,overweight


In [125]:

#Here I renamed the column "sex" to "gender"

filtered_df = filtered_df.rename(columns={"sex": "gender"})
filtered_df

,age group,age,gender,children,has children,smoker,region,bmi,BMI Category
0,60s,19,female,0,no,yes,southwest,27.900,overweight
1,teen,18,male,1,yes,no,southeast,33.770,obese
2,20s,28,male,3,yes,no,southeast,33.000,obese
3,30s,33,male,0,no,no,northwest,22.705,normal
4,30s,32,male,0,no,no,northwest,28.880,overweight
...,...,...,...,...,...,...,...,...,...
1333,50s,50,male,3,yes,no,northwest,30.970,obese
1334,teen,18,female,0,no,no,northeast,31.920,obese
1335,teen,18,female,0,no,no,southeast,36.850,obese
1336,20s,21,female,0,no,no,southwest,25.800,overweight


In [132]:
#Here I have sorted the data based on BMI Category, and reset the index

filtered_df.sort_values('BMI Category', ascending=True).reset_index(drop=True)

,age group,age,gender,children,has children,smoker,region,bmi,BMI Category
0,40s,46,female,2,yes,no,northwest,19.950,normal
1,60s,19,female,0,no,no,northwest,22.515,normal
2,teen,18,female,0,no,yes,northeast,21.660,normal
3,50s,55,male,1,yes,no,southwest,21.500,normal
4,40s,43,female,2,yes,yes,northeast,20.045,normal
...,...,...,...,...,...,...,...,...,...
1313,20s,21,male,0,no,no,northeast,27.360,overweight
1314,30s,34,female,1,yes,no,northwest,26.410,overweight
1315,40s,47,female,1,yes,no,northwest,29.545,overweight
1316,30s,36,male,3,yes,no,northeast,28.880,overweight


In [131]:
filtered_df.to_csv('analysis_pipeline.csv', index=False)